In [1]:
%load_ext autoreload
%autoreload 2

import gymnasium as gym


from src.actor_critic import Mlp, ActorCritic
from src.trainer import PPOModule, PPOModuleConfig
import lightning as pl

In [2]:
from stable_baselines3 import DQN, PPO
# Create environment
env = gym.make("LunarLander-v3", render_mode="rgb_array")

# Instantiate the agent
model = PPO("MlpPolicy", env, verbose=1)
model.policy

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=8, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=8, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=4, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=True)
)

In [3]:
# Initialise the environment
env = gym.make("LunarLander-v3")

# make the actor critic model
d = env.observation_space.shape[0]
n_a = env.action_space.n
hidden_dims = 64

# feature_extractor = Mlp(input_dims=d, output_dims=hidden_dims, hidden_dims=hidden_dims, n_layers=3)
actor = Mlp(input_dims=d, output_dims=n_a, hidden_dims=hidden_dims, n_layers=1, activation="silu")
critic = Mlp(input_dims=d, output_dims=1, hidden_dims=hidden_dims, n_layers=1, activation="silu")

model = ActorCritic(actor_net=actor, critic=critic, feature_extractor=None)


config = PPOModuleConfig()
trainer = pl.Trainer(max_epochs=100,)
module = PPOModule(actor_critic=model, env=env, config=config)

trainer.fit(module)
# module

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type        | Params | Mode 
-----------------------------------------------------
0 | actor_critic | ActorCritic | 9.8 K  | train
-----------------------------------------------------
9.8 K     Trainable params
0         Non-trainable params
9.8 K     Total params
0.039     Total estimated model params size (MB)
14        Modules in train mode
0         Modules in eval mode
/Users/nicholasfranklin/miniconda3/envs/state_inference/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/nicholasfranklin/miniconda3/envs/state_inference/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of t

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=25` reached.


In [ ]:
# Initialise the environment
env = gym.make("LunarLander-v3", render_mode="human")

# Reset the environment to generate the first observation
observation, info = env.reset(seed=42)
for _ in range(5000):
    # this is where you would insert your policy
    action = module.predict(observation)

    # step (transition) through the environment with the action
    # receiving the next observation, reward and if the episode has terminated or truncated
    observation, reward, terminated, truncated, info = env.step(action)

    # If the episode has ended then we can reset to start a new episode
    if terminated or truncated:
        observation, info = env.reset()

env.close()

2025-02-06 18:51:32.645 python[70451:2994810] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-06 18:51:32.645 python[70451:2994810] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [ ]:
actor